In [1]:
import maliput

In [2]:
help(maliput.api)

Help on module maliput.api in maliput:

NAME
    maliput.api

CLASSES
    pybind11_builtins.pybind11_object(builtins.object)
        InertialPosition
        Junction
        Lane
        LaneId
        LanePosition
        LanePositionResult
        RoadGeometry
        RoadGeometry.IdIndex
        RoadGeometryId
        RoadNetwork
        RoadPosition
        Rotation
        Segment
    
    class InertialPosition(pybind11_builtins.pybind11_object)
     |  Method resolution order:
     |      InertialPosition
     |      pybind11_builtins.pybind11_object
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  __init__(...)
     |      __init__(self: maliput.api.InertialPosition, x: float, y: float, z: float) -> None
     |  
     |  xyz(...)
     |      xyz(self: maliput.api.InertialPosition) -> maliput::math::Vector3
     |  
     |  ----------------------------------------------------------------------
     |  Static methods inherited from pybind11_bui

In [3]:
help(maliput.math)

Help on module maliput.math in maliput:

NAME
    maliput.math

CLASSES
    pybind11_builtins.pybind11_object(builtins.object)
        Quaternion
        RollPitchYaw
        Vector3
        Vector4
    
    class Quaternion(pybind11_builtins.pybind11_object)
     |  Method resolution order:
     |      Quaternion
     |      pybind11_builtins.pybind11_object
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  __init__(...)
     |      __init__(self: maliput.math.Quaternion, arg0: float, arg1: float, arg2: float, arg3: float) -> None
     |  
     |  coeffs(...)
     |      coeffs(self: maliput.math.Quaternion) -> maliput.math.Vector4
     |  
     |  w(...)
     |      w(self: maliput.math.Quaternion) -> float
     |  
     |  x(...)
     |      x(self: maliput.math.Quaternion) -> float
     |  
     |  y(...)
     |      y(self: maliput.math.Quaternion) -> float
     |  
     |  z(...)
     |      z(self: maliput.math.Quaternion) -> float
     |  
    

In [4]:
manager = maliput.plugin.MaliputPluginManager()
manager.ListPlugins()


['maliput_malidrive']

In [5]:
# https://shiny-fiesta-6b790eb7.pages.github.io/from_doxygen/html/deps/maliput_malidrive/html/group__road__network__configuration__builder__keys.html
# 
road_network_options = {
    "opendrive_file": "/mnt/maliput/workspaces/python_exploration/src/maliput_malidrive/maliput_malidrive/resources/figure8_trafficlights/figure8_trafficlights.xodr"
}


In [6]:
network1 = maliput.plugin.create_road_network_from_plugin("maliput_malidrive", road_network_options)

In [7]:
help(network1.road_geometry())
help(network1.road_geometry().junction(0))
help(network1.road_geometry().junction(0).segment(0))
help(network1.road_geometry().junction(0).segment(0).lane(0))
help(network1.road_geometry().junction(0).segment(0).lane(0).id())
geometry = network1.road_geometry()

Help on RoadGeometry in module maliput.api object:

class RoadGeometry(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      RoadGeometry
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  ById(...)
 |      ById(self: maliput.api.RoadGeometry) -> maliput::api::RoadGeometry::IdIndex
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  id(...)
 |      id(self: maliput.api.RoadGeometry) -> maliput.api.RoadGeometryId
 |  
 |  junction(...)
 |      junction(self: maliput.api.RoadGeometry, arg0: int) -> maliput::api::Junction
 |  
 |  num_junctions(...)
 |      num_junctions(self: maliput.api.RoadGeometry) -> int
 |  
 |  ----------------------------------------------------------------------
 |  Static methods inherited from pybind11_builtins.pybind11_object:
 |  
 |  __new__(*args, **kwargs) from pybind11_builtins.pybind11_type
 |      Create and re

In [9]:
# Iterate all items.
print("Geometry. Num junctions: {}".format(geometry.num_junctions()))
for junction_index in range(geometry.num_junctions()):
    junction = geometry.junction(junction_index)
    # help(junction)
    print("  Junction {}: Segments: {}".format(junction.id().string(), junction.num_segments()))
    for segment_index in range(junction.num_segments()):
        segment = junction.segment(segment_index)
        print("    Segment {}: Lanes: {}".format(segment.id().string(), segment.num_lanes()))
        for lane_index in range(segment.num_lanes()):
            lane = segment.lane(lane_index)
            print("    Lane {}: length: {}".format(lane.id().string(), lane.length()))
            

Geometry. Num junctions: 11


AttributeError: 'maliput.api.Junction' object has no attribute 'id'

In [10]:
# Sample so I can get help.
p_inertial = maliput.api.InertialPosition(0,0,0)
road_position_result = geometry.ToRoadPosition(p_inertial)
road_position = road_position_result.road_position
help(p_inertial)
help(road_position_result)
help(road_position)
help(road_position.lane)


AttributeError: 'maliput.api.RoadGeometry' object has no attribute 'ToRoadPosition'

In [11]:
# Perform some to road queries, to lane queries, and back.
positions = [
    [0, 0, 0],
    [0, 1, 0],
    [0, 25, 0]
]
for position in positions:
    p_inertial = maliput.api.InertialPosition(*position)
    # Would be nice to have __repr__
    print("inertial {}, {}, {}".format(p_inertial.xyz().x(), p_inertial.xyz().y(), p_inertial.xyz().z()))
    road_position_result = geometry.ToRoadPosition(p_inertial)
    road_position = road_position_result.road_position
    print(
        "road position {}, ({},{},{})".format(
            str(road_position.lane.id().string()),
            road_position.pos.srh().x(),
            road_position.pos.srh().y(),
            road_position.pos.srh().z()
        )
    )
    # back to inertial
    p_roundtrip = road_position.lane.ToInertialPosition(road_position.pos)
    print("inertial roundtrip {}, {}, {}".format(p_roundtrip.xyz().x(), p_roundtrip.xyz().y(), p_roundtrip.xyz().z()))
    
    
    print()

inertial 0.0, 0.0, 0.0


AttributeError: 'maliput.api.RoadGeometry' object has no attribute 'ToRoadPosition'

In [12]:
# Now load a road network with rulebooks.
# https://shiny-fiesta-6b790eb7.pages.github.io/from_doxygen/html/deps/maliput_malidrive/html/group__road__network__configuration__builder__keys.html
# 
road_network_options2 = {
    "opendrive_file": "/mnt/maliput/workspaces/python_exploration/src/maliput_malidrive/maliput_malidrive/resources/figure8_trafficlights/figure8_trafficlights.xodr",
    "road_rule_book": "/mnt/maliput/workspaces/python_exploration/src/maliput_malidrive/maliput_malidrive/resources/figure8_trafficlights/figure8_trafficlights.yaml",
}
road_network_options2["traffic_light_book"] = road_network_options2["road_rule_book"]
road_network_options2["phase_ring_book"] = road_network_options2["road_rule_book"]
road_network_options2["intersection_book"] = road_network_options2["road_rule_book"]


In [13]:
network2 = maliput.plugin.create_road_network_from_plugin("maliput_malidrive", road_network_options2)

In [14]:
help(network2)

Help on RoadNetwork in module maliput.api object:

class RoadNetwork(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      RoadNetwork
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  road_geometry(...)
 |      road_geometry(self: maliput.api.RoadNetwork) -> maliput::api::RoadGeometry
 |  
 |  ----------------------------------------------------------------------
 |  Static methods inherited from pybind11_builtins.pybind11_object:
 |  
 |  __new__(*args, **kwargs) from pybind11_builtins.pybind11_type
 |      Create and return a new object.  See help(type) for accurate signature.

